In [97]:
import os
import json
import yfinance as yf
import pandas as pd
import numpy as np
from pandas.tseries.offsets import BDay

PERIOD = 3
LOT = 25000
GAIN_CAP = 0.1
HOLDING_PERIOD_CAP = 5  #business days
#Trip = ENTRY,EXIT
#Action = BUY,SELL
#Status = FILLED, PENDING, CANCELED, FORCED
blotter = pd.DataFrame(
    columns=['Date', 'Symbol', 'Trip', 'Action', 'Price', 'Size', 'Status'])

# Historical Market Data

In [98]:
file_path = os.getenv("ITA_DATA_PATH")
hist = pd.read_csv(f'{file_path}\data.csv', parse_dates=True, index_col='Date')
hist = hist.iloc[::-1]
hist['log_ret_AMZN'] = np.log(hist['amzn_Close']) - np.log(hist['amzn_Close'].shift(1))
hist['log_ret_WMT'] = np.log(hist['wmt_Close']) - np.log(hist['wmt_Close'].shift(1))
hist['corr_coef'] = hist['amzn_Close'].rolling(PERIOD).corr(hist['wmt_Close'])
hist.head(10)

,wmt_Open,wmt_High,wmt_Low,wmt_Close,wmt_Volume,amzn_Open,amzn_High,amzn_Low,amzn_Close,amzn_Volume,log_ret_AMZN,log_ret_WMT,corr_coef
Date,,,,,,,,,,,,,
2017-01-03,69.24,69.24,68.05,68.66,10473162,757.92,758.76,747.7,753.67,3521066,NaN,NaN,NaN
2017-01-04,68.66,69.63,68.6,69.06,7917952,758.39,759.68,754.2,757.18,2510526,0.00464639946359835,0.005808903932300602,NaN
2017-01-05,68.43,69.33,68.12,69.21,7099170,761.55,782.4,760.26,780.45,5830068,0.030269670518983993,0.0021696688919252693,0.7903801104055947
2017-01-06,68.41,68.5,68.01,68.26,9491115,782.36,799.44,778.48,795.99,5986234,0.01971594654697828,-0.01382141737521092,-0.7070525511404454
2017-01-09,68.33,68.8,68.32,68.71,8685232,798.0,801.77,791.77,796.92,3446109,0.001167674389222384,0.00657080556442402,-0.8559035132754906
2017-01-10,68.64,69.27,68.22,68.23,10256492,796.6,798.0,789.54,795.9,2558369,-0.001280747528818793,-0.007010398215652636,0.9997130545224221
2017-01-11,68.2,68.62,68.19,68.53,6697593,793.66,799.5,789.51,799.02,2992791,0.003912426930305024,0.004387254770399807,0.452612624589008
2017-01-12,68.36,68.56,67.92,67.97,6544856,800.31,814.13,799.5,813.64,4873922,0.018132030992177306,-0.008205174239185098,-0.7449849425605057
2017-01-13,67.98,68.02,67.08,67.13,10012519,814.32,821.65,811.4,817.14,3791945,0.004292431074635061,-0.012435393407886686,-0.8980534734268214


# Get Position Direction

In [99]:
# This is another logic to decide if we should go long or short
# It's only based on the number of negative returns in a period of time (no tie-breaker)
def get_position_direction(slice_hist):
    amzn_neg_count = np.sum((slice_hist['log_ret_AMZN'].values < 0))
    wmt_neg_count = np.sum((slice_hist['log_ret_WMT'].values < 0))
    amzn_direction = 'BUY' if (amzn_neg_count >= wmt_neg_count) else 'SELL'
    wmt_direction = 'SELL' if (amzn_direction == 'BUY') else 'BUY'
    return {'AMZN': amzn_direction, 'WMT': wmt_direction}

In [100]:
# This logic decides if we should go long or short
# It's based on the volume of stocks that had negative returns (it's tie-breaker)
def get_position_direction2(slice_hist):
    amzn_total_volume = slice_hist['amzn_Volume'].sum()
    amz_negative_volume = np.where(slice_hist['log_ret_AMZN'] < 0, slice_hist['amzn_Volume'], 0).sum()
    amz_negative_pct = amz_negative_volume / amzn_total_volume

    wmt_total_volume = slice_hist['wmt_Volume'].sum()
    wmt_negative_volume = np.where(slice_hist['log_ret_WMT'] < 0, slice_hist['wmt_Volume'], 0).sum()
    wmt_negative_pct = wmt_negative_volume / wmt_total_volume

    amzn_direction = 'BUY' if (amz_negative_pct >= wmt_negative_pct) else 'SELL'
    wmt_direction = 'SELL' if (amzn_direction == 'BUY') else 'BUY'

    return {'AMZN': amzn_direction, 'WMT': wmt_direction}

# Create Entry Trades

In [101]:
def create_entry_trades(biz_date):
    from_date = biz_date - BDay(PERIOD - 1)
    to_date = biz_date
    slice_hist = hist.loc[from_date: to_date, :]

    trade_date = hist[hist.index > biz_date].index[0]  #get next date
    pos_direction = get_position_direction2(slice_hist)
    amzn_open_price = hist.loc[trade_date]['amzn_Open']
    amzn_size = LOT / amzn_open_price
    wmt_open_price = hist.loc[trade_date]['wmt_Open']
    wmt_size = LOT / wmt_open_price

    amzn_entry_trade = {
        "Date": trade_date,
        "Symbol": 'AMZN',
        "Trip": 'ENTRY',
        "Action": pos_direction['AMZN'],
        "Price": amzn_open_price,
        "Size": round(amzn_size, 4),  #rounding based on IB's minimum fractional size
        "Status": 'FILLED'
    }
    wmt_entry_trade = {
        "Date": trade_date,
        "Symbol": 'WMT',
        "Trip": 'ENTRY',
        "Action": pos_direction['WMT'],
        "Price": wmt_open_price,
        "Size": round(wmt_size, 4),
        "Status": 'FILLED'
    }
    return pd.DataFrame([amzn_entry_trade, wmt_entry_trade])

# Create Exit Trades

In [102]:
#The exit trades are LMT orders created right after entry trades with "Date" as the expiration date
def create_exit_trade(entry_trades_lcl, symbol):
    entry_data = entry_trades_lcl[entry_trades['Symbol'] == symbol].reset_index()
    entry_date = entry_data.at[0, 'Date']
    exit_date = entry_date + BDay(HOLDING_PERIOD_CAP)

    entry_action = entry_data.at[0, 'Action']
    exit_action = 'SELL' if entry_action == 'BUY' else 'BUY'

    entry_price = entry_data.at[0, 'Price']
    exit_price = None
    if exit_action == 'SELL':
        exit_price = entry_price * (1 + GAIN_CAP)
    else:
        exit_price = entry_price * (1 - GAIN_CAP)

    exit_size = entry_data.at[0, 'Size']

    return {
        "Date": exit_date,
        "Symbol": symbol,
        "Trip": 'EXIT',
        "Action": exit_action,
        "Price": round(exit_price, 2),
        "Size": exit_size,
        "Status": 'PENDING'
    }

In [103]:
def create_exit_trades(entry_trades_lcl):
    amzn_exit_trade = create_exit_trade(entry_trades_lcl, 'AMZN')
    wmt_exit_trade = create_exit_trade(entry_trades_lcl, 'WMT')
    return pd.DataFrame([amzn_exit_trade, wmt_exit_trade])

In [104]:
# The forced trades are MKT orders created when the exit trades are not filled within the holding period cap.
# It uses the Close Price of the next day following the exit trade's date
def create_forced_trade(biz_date, symbol, action, size, market_data):
    column_prefix = symbol.lower()
    close_price_column_name = f"{column_prefix}_Close"
    return {
        "Date": biz_date,
        "Symbol": symbol,
        "Trip": 'EXIT',
        "Action": action,
        "Price": market_data[close_price_column_name],
        "Size": size,
        "Status": 'FORCED'
    }

In [105]:
def should_force_close_position(position, biz_date):
    if position['Date'] < biz_date:
        return True

In [106]:
def should_close_position(position, biz_date):
    column_prefix = position['Symbol'].lower()
    high_price_column_name = f"{column_prefix}_High"
    low_price_column_name = f"{column_prefix}_Low"
    if position['Action'] == 'SELL' and hist.loc[biz_date][high_price_column_name] >= position['Price']:
        return True
    if position['Action'] == 'BUY' and hist.loc[biz_date][low_price_column_name] <= position['Price']:
        return True
    return False

# Run Back Test

In [107]:
for index, today_market_data in hist.iterrows():
    business_date = index
    pending_exit_trades = blotter[(blotter['Trip'] == 'EXIT') & (blotter['Status'] == 'PENDING')]
    current_position_status = 'CLOSED' if  pending_exit_trades.empty else 'OPEN'

    if current_position_status == 'CLOSED' and today_market_data['corr_coef'] < 0:
        entry_trades = create_entry_trades(business_date)
        blotter = pd.concat([blotter, entry_trades], ignore_index=True)

        exit_trades = create_exit_trades(entry_trades)
        blotter = pd.concat([blotter, exit_trades], ignore_index=True)
    elif current_position_status == 'OPEN':
        for i, pending_trade in pending_exit_trades.iterrows():
            if should_force_close_position(pending_trade, business_date):
                blotter.at[i, 'Status'] = 'CANCELED'
                forced_trade = create_forced_trade(business_date,
                                                   pending_trade['Symbol'],
                                                   pending_trade['Action'],
                                                   pending_trade['Size'],
                                                   today_market_data)
                blotter = pd.concat([blotter, pd.DataFrame([forced_trade])], ignore_index=True)
            elif should_close_position(pending_trade, business_date):
                blotter.at[i, 'Status'] = 'FILLED'

# Result & Stats

In [113]:
blotter

,Date,Symbol,Trip,Action,Price,Size,Status
0,2017-01-09 00:00:00,AMZN,ENTRY,SELL,798.0,31.3283,FILLED
1,2017-01-09 00:00:00,WMT,ENTRY,BUY,68.33,365.8715,FILLED
2,2017-01-16 00:00:00,AMZN,EXIT,BUY,718.2,31.3283,CANCELED
3,2017-01-16 00:00:00,WMT,EXIT,SELL,75.16,365.8715,CANCELED
4,2017-01-17 00:00:00,AMZN,EXIT,BUY,809.72,31.3283,FORCED
5,2017-01-17 00:00:00,WMT,EXIT,SELL,68.42,365.8715,FORCED
6,2017-01-19 00:00:00,AMZN,ENTRY,BUY,810.0,30.8642,FILLED
7,2017-01-19 00:00:00,WMT,ENTRY,SELL,68.13,366.9455,FILLED
8,2017-01-26 00:00:00,AMZN,EXIT,SELL,891.0,30.8642,CANCELED
9,2017-01-26 00:00:00,WMT,EXIT,BUY,61.32,366.9455,CANCELED


In [109]:
def calculate_gain_loss(symbol):
    results = blotter[(blotter['Status'] != 'CANCELED') & (blotter['Symbol'] == symbol)].copy()
    results['Total_Price'] = results['Price'] * results['Size']
    sells = results[results['Action'] == 'SELL'].copy()
    buys = results[results['Action'] == 'BUY'].copy()

    total_sales = sells['Total_Price'].sum()
    total_purchases = buys['Total_Price'].sum()
    gain_loss = total_sales - total_purchases

    print(f"\n******  {symbol}  ******")
    print(f'Total Sales: ${round(total_sales, 2):,}')
    print(f'Total Purchases: ${round(total_purchases, 2):,}')
    print(f'Gain or Loss: ${round(gain_loss, 2):,}')
    return gain_loss


print(f"Entry Orders: {blotter[(blotter['Trip'] == 'ENTRY') & (blotter['Status'] == 'FILLED')].shape[0]}")
print(f"Filled Exit Orders: {blotter[(blotter['Trip'] == 'EXIT') & (blotter['Status'] == 'FILLED')].shape[0]}")
print(f"Forced Exit Orders: {blotter[(blotter['Trip'] == 'EXIT') & (blotter['Status'] == 'FORCED')].shape[0]}")
amz_gain_loss = calculate_gain_loss('AMZN')
wmt_gain_loss = calculate_gain_loss('WMT')
total_gain_loss = amz_gain_loss + wmt_gain_loss
time_period = hist.last_valid_index() - hist.first_valid_index()
years = round(time_period.days / 365.2425, 2)
total_gain_loss_per_year = total_gain_loss / years
print(f"\nYears: {years}")
print(f'Total Gain or Loss: ${round(total_gain_loss, 2):,}')
print(f"Total Gain or Loss Per Year: ${round(total_gain_loss_per_year, 2):,}")

Entry Orders: 264
Filled Exit Orders: 9
Forced Exit Orders: 255

******  AMZN  ******
Total Sales: $3,318,757.29
Total Purchases: $3,302,575.22
Gain or Loss: $16,182.06

******  WMT  ******
Total Sales: $3,308,183.56
Total Purchases: $3,301,093.78
Gain or Loss: $7,089.78

Years: 5.29
Total Gain or Loss: $23,271.84
Total Gain or Loss Per Year: $4,399.21
